In [1]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm #para poder ver las barritas de % cargandose cuando va iterando
from PIL import Image as im

#PyTorch
import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import TensorDataset, DataLoader, random_split

import torch.nn as nn
import torch.nn.functional as F

#Confusion matrix
from sklearn.metrics import confusion_matrix
import seaborn as sn

#Optuna
import optuna
from optuna.integration import PyTorchLightningPruningCallback

In [2]:
#Para usar solo 24 cores que me dijo NW

torch.set_num_threads(24)

In [ ]:
# Set seeds for reproducibility (PyTorch, Python, Numpy)
matricola = 2013031
torch.manual_seed(matricola)
random.seed(matricola)
np.random.seed(matricola)

In [3]:
# Function to save the model 
def saveModel(model): 
    path = "./Modelos_Fiteados/REG/NUEVAS/2C_L1.pth" 
    torch.save(model.state_dict(), path)

# DATASET

In [4]:
#Bajo el MNIST
datatrain = torchvision.datasets.MNIST('./', train=True, download=True)
datatest = torchvision.datasets.MNIST('./', train=False, download=True)

m=len(datatrain)

train_data, val_data = random_split(datatrain, [int(m-m*0.2), int(m*0.2)])

#Lo guardo en listas en vez de tensores
X_train = [x[0] for x in train_data]
y_train = [x[1] for x in train_data]

X_val = [x[0] for x in val_data]
y_val = [x[1] for x in val_data]

X_test = [x[0] for x in datatest]
y_test = [x[1] for x in datatest]


RX_train = [0]*(20*len(X_train))
RY_train = [0]*(20*len(X_train))

RX_val = [0]*(20*len(X_val))
RY_val = [0]*(20*len(X_val))

RX_test = [0]*(20*len(X_test))
RY_test = [0]*(20*len(X_test))

In [5]:
rotador = torchvision.transforms.functional.rotate

In [6]:
for i in range(len(X_train)):
    for j in range(20):
        ang = random.uniform(-60,60)
        RX_train[i*20+j]=rotador(X_train[i],ang)
        RY_train[i*20+j]=ang
        
for i in range(len(X_val)):
    for j in range(20):
        ang = random.uniform(-60,60)
        RX_val[i*20+j]=rotador(X_val[i],ang)
        RY_val[i*20+j]=ang
    
for i in range(len(X_test)):
    for j in range(20):
        ang = random.uniform(-60,60)
        RX_test[i*20+j]=rotador(X_test[i],ang)
        RY_test[i*20+j]=ang

In [ ]:
print(len(RX_train), len(RX_val), len(RX_test))

In [ ]:
#Veamos si estan masomenos distribuidos
plt.hist(RY_test,10)
plt.title('Histograma de labels')
plt.xlabel('label')
plt.ylabel('Frecuencia')

plt.show() #dibujamos el histograma

In [ ]:
# Plot some samples
fig, axs = plt.subplots(5, 5, figsize=(8,8))
i = 0
for ax in axs.flatten():
    ax.imshow(np.array(RX_train[i*20]), cmap='gist_gray')
    ax.set_title(f'Angle: {RY_train[i*20]}')#{round((RY_train[i*20]),2)}')
    ax.set_xticks([])
    ax.set_yticks([])
    i +=1
#plt.savefig('./Modelos_Fiteados/Imagenes/REG/.png', format='png')
plt.tight_layout()

In [7]:
#hago la transformacion a tensores
to_tensor = torchvision.transforms.ToTensor()

#los convierto a tensores con la transformacion
tensor_RX_train = [to_tensor(RX_train[i]).unsqueeze(0) for i in range(len(RX_train))]
tensor_RX_train = torch.cat(tensor_RX_train)

tensor_RY_train = torch.Tensor(RY_train).float()
###
tensor_RX_val = [to_tensor(RX_val[i]).unsqueeze(0) for i in range(len(RX_val))]
tensor_RX_val = torch.cat(tensor_RX_val)

tensor_RY_val = torch.Tensor(RY_val).float()
###
tensor_RX_test = [to_tensor(RX_test[i]).unsqueeze(0) for i in range(len(RX_test))]
tensor_RX_test = torch.cat(tensor_RX_test)

tensor_RY_test = torch.Tensor(RY_test).float()

#creo los datatrian y datatest
tensor_datatrain = TensorDataset(tensor_RX_train, tensor_RY_train)
train_loader = DataLoader(tensor_datatrain, batch_size=5000, shuffle=True)

tensor_dataval = TensorDataset(tensor_RX_val, tensor_RY_val)
valid_loader = DataLoader(tensor_dataval, batch_size=5000, shuffle=True)

tensor_datatest = TensorDataset(tensor_RX_test, tensor_RY_test)
test_loader = DataLoader(tensor_datatest, batch_size=1)

### Red 3C

In [ ]:
class Net(nn.Module):
    def __init__(self,nf):
        super(Net,self).__init__()
#         ariquitecture
        self.net = nn.Sequential(nn.Conv2d(in_channels  = 1,        # First convolutional layer
                                           out_channels = nf,
                                           kernel_size  = 3,
                                           stride       = 2,
                                           padding      = 1),
                                 nn.ReLU(True),                     # OUT  [nf X 14 X 14]
                                 nn.Conv2d(in_channels  = nf,        # Second convolutional layer
                                           out_channels = 2 * nf,
                                           kernel_size  = 3,
                                           stride       = 2,
                                           padding      = 1),
                                 nn.ReLU(True),                      # OUT [2nf X 7 X 7]
                                 nn.Conv2d(in_channels  = 2 * nf,     # Third convolutional layer
                                           out_channels = 4 * nf,
                                           kernel_size  = 3,
                                           stride       = 2,
                                           padding      = 0),
                                 nn.ReLU(True),                     # OUT [4nf X 3 x 3]
                                 nn.Flatten(start_dim=1),            # Flatten layer
                                 nn.Linear(in_features  = ((4*nf)*3*3),
                                           out_features = 128),
                                 nn.ReLU(True),
                                 nn.Linear(in_features  = 128,
                                           out_features = 64),
                                 nn.ReLU(True),
                                 nn.Dropout(0.2),
                                 nn.Linear(in_features  = 64,
                                           out_features = 1,
                                           bias=False)
                                )
        
    def forward(self, x, mode):
        if (mode == "Train"):
            self.net.train()
        elif (mode == "Test"):
            self.net.eval()            
        # Apply encoder decoder
        x = self.net(x)
        return x
    
    ###Training function
    def train_epoch(self,device,dataloader, loss_fn, optimizer, verbose = True):
        """
        This function train the network for one epoch
        """
        # Train
        train_loss = []
        for data_batched, label_batched in dataloader:
            # Move data to device
            data_batched = data_batched.to(device)
            label_batched = label_batched.to(device)
            # process
            processed = self.forward(data_batched,"Train")[:,0] #dimension del output para que la loss la entienda
            # Compute loss
            loss = loss_fn(processed, label_batched)
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            #Updata weights
            optimizer.step()
            #Save trai loss for this batch
            loss_batch = loss.detach().cpu().numpy()
            train_loss.append(loss_batch)
        #Save the average train loss
        train_loss = np.mean(train_loss)
#         if verbose: print(f"AVERAGE TRAIN LOSS: {train_loss}")

        return train_loss
    
    ### Testing function
    def test_epoch(self,device, dataloader, loss_fn, verbose= True):
        """
        This function test the network performance for one epoch of training
        """
        test_loss = []
        # Discable gradient tracking
        with torch.no_grad():
            for data_batched, label_batched in dataloader:
                # Move data to device
                data_batched = data_batched.to(device)
                label_batched = label_batched.to(device)
                # process
                processed = self.forward(data_batched,"Test")[:,0]
                # Compute loss
                loss = loss_fn(processed, label_batched)
                 #Save test loss for this batch
                loss_batch = loss.detach().cpu().numpy()
                test_loss.append(loss_batch)
            #Save the average train loss
            test_loss = np.mean(test_loss)
#             if verbose: print(f"AVERAGE TEST LOSS: {test_loss}")

        return test_loss
    
    def training_cycle(self, device, training_data, test_data, loss_fn, optim, num_epochs,
                       keep_model=False, verbose= True):
        """
        This function train the network for a desired number of epochs it also test the network 
        reconstruction performance and make plots comparing the input image and the reconstructed one
        every 5 epochs.
        """
        
        diz_loss = {'train_loss_1':[], 'train_loss_2':[], 'val_loss':[]}
        for epoch in tqdm(range(num_epochs)):
            train_loss_1 =train_epoch(CLASIF,
                            device,
                            train_loader,
                            loss_fn,
                            optimizer)
            train_loss_2 = test_epoch(CLASIF,
                          device,
                          train_loader,
                          loss_fn)
            val_loss = test_epoch(CLASIF,
                          device,
                          valid_loader,
                          loss_fn)
            print('\n EPOCH {}/{} \t train loss_1 {} \t train loss_2 {} \t val loss {}'.format(epoch + 1, num_epochs,train_loss_1,train_loss_2,val_loss))
            diz_loss['train_loss_1'].append(train_loss_1)
            diz_loss['train_loss_2'].append(train_loss_2)
            diz_loss['val_loss'].append(val_loss)
            
            
#         #I keep track of losses for plots
#         train_loss_1 = []
#         train_loss_2 = []
#         test_loss    = []
#         i = 0
#         for epoch in tqdm(range(num_epochs)):
#             if verbose: print('EPOCH %d/%d' % (epoch + 1, num_epochs))
#             ### Training (use the training function)
#             tr_l = self.train_epoch(
#                 device=device, 
#                 dataloader=training_data, 
#                 loss_fn=loss_fn, 
#                 optimizer=optim,
#                 verbose = verbose)
#             train_loss_1.append(tr_l)
#             ### training_2  (use the training for loss)
#             t_l = self.test_epoch(
#                 device=device, 
#                 dataloader=training_data, 
#                 loss_fn=loss_fn,
#                 verbose = verbose)
#             train_loss_2.append(t_l)
#             ### Validation  (use the testing function)
#             t_l = self.test_epoch(
#                 device=device, 
#                 dataloader=test_data, 
#                 loss_fn=loss_fn,
#                 verbose = verbose)
#             test_loss.append(t_l)

        return diz_loss['train_loss_1'], diz_loss['train_loss_2'], diz_loss['val_loss']
# train_loss_1, train_loss_2, test_loss

### Red 2C

In [8]:
class Net(nn.Module):    
    def __init__(self,nf):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, nf, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(nf, nf, 3)
        self.fc1 = nn.Linear(nf*12*12, 128)
        self.fc2 = nn.Linear(128, 1)
        self.drop = nn.Dropout(0.25)
        self.soft = nn.Softmax()

    def forward(self, x, mode):
        if (mode == "Train"):
            self.train()
        elif (mode == "Test"):
            self.eval()
        
        # Apply net
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.drop(x)
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.drop(x)
        x = self.fc2(x)
        return x
    
    ### Training function
    def train_epoch(self,device,dataloader, loss_fn, optimizer, verbose = True):
        """
        This function train the network for one epoch
        """
        # Train
        train_loss = []
        for data_batched, label_batched in dataloader:
            # Move data to device
            data_batched = data_batched.to(device)
            label_batched = label_batched.to(device)
            # process
            processed = self.forward(data_batched,"Train")[:,0] #dimension del output para que la loss la entienda
            # Compute loss
            loss = loss_fn(processed, label_batched)
            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            #Updata weights
            optimizer.step()
            #Save trai loss for this batch
            loss_batch = loss.detach().cpu().numpy()
            train_loss.append(loss_batch)
        #Save the average train loss
        train_loss = np.mean(train_loss)
#         if verbose: print(f"AVERAGE TRAIN LOSS: {train_loss}")

        return train_loss
    
    ### Testing function
    def test_epoch(self,device, dataloader, loss_fn, verbose= True):
        """
        This function test the network performance for one epoch of training
        """
        test_loss = []
        # Discable gradient tracking
        with torch.no_grad():
            for data_batched, label_batched in dataloader:
                # Move data to device
                data_batched = data_batched.to(device)
                label_batched = label_batched.to(device)
                # process
                processed = self.forward(data_batched,"Test")[:,0]
                # Compute loss
                loss = loss_fn(processed, label_batched)
                 #Save test loss for this batch
                loss_batch = loss.detach().cpu().numpy()
                test_loss.append(loss_batch)
            #Save the average train loss
            test_loss = np.mean(test_loss)
#             if verbose: print(f"AVERAGE TEST LOSS: {test_loss}")

        return test_loss
    
    def training_cycle(self, device, training_data, test_data, loss_fn, optim, num_epochs,
                       keep_model=False, verbose= True):
        """
        This function train the network for a desired number of epochs it also test the network 
        reconstruction performance and make plots comparing the input image and the reconstructed one
        every 5 epochs.
        """
        
        diz_loss = {'train_loss_1':[], 'train_loss_2':[], 'val_loss':[]}
        for epoch in tqdm(range(num_epochs)):
            train_loss_1 =train_epoch(CLASIF,
                            device,
                            train_loader,
                            loss_fn,
                            optimizer)
            train_loss_2 = test_epoch(CLASIF,
                          device,
                          train_loader,
                          loss_fn)
            val_loss = test_epoch(CLASIF,
                          device,
                          valid_loader,
                          loss_fn)
            print('\n EPOCH {}/{} \t train loss_1 {} \t train loss_2 {} \t val loss {}'.format(epoch + 1, num_epochs,train_loss_1,train_loss_2,val_loss))
            diz_loss['train_loss_1'].append(train_loss_1)
            diz_loss['train_loss_2'].append(train_loss_2)
            diz_loss['val_loss'].append(val_loss)
            
            
#         #I keep track of losses for plots
#         train_loss_1 = []
#         train_loss_2 = []
#         test_loss    = []
#         i = 0
#         for epoch in tqdm(range(num_epochs)):
#             if verbose: print('EPOCH %d/%d' % (epoch + 1, num_epochs))
#             ### Training (use the training function)
#             tr_l = self.train_epoch(
#                 device=device, 
#                 dataloader=training_data, 
#                 loss_fn=loss_fn, 
#                 optimizer=optim,
#                 verbose = verbose)
#             train_loss_1.append(tr_l)
#             ### training_2  (use the training for loss)
#             t_l = self.test_epoch(
#                 device=device, 
#                 dataloader=training_data, 
#                 loss_fn=loss_fn,
#                 verbose = verbose)
#             train_loss_2.append(t_l)
#             ### Validation  (use the testing function)
#             t_l = self.test_epoch(
#                 device=device, 
#                 dataloader=test_data, 
#                 loss_fn=loss_fn,
#                 verbose = verbose)
#             test_loss.append(t_l)

        return diz_loss['train_loss_1'], diz_loss['train_loss_2'], diz_loss['val_loss']
# train_loss_1, train_loss_2, test_loss

In [9]:
max_epochs = 30

#RED CON 3C
# nf = 8

#RED CON 2C
nf = 64

# Inicializamos la red
model = Net(nf)

In [10]:
#Definimos la loss function
# loss_fn = nn.MSELoss()
loss_fn = nn.L1Loss()

# Definimos el optimizador
optimizer = torch.optim.Adam(model.parameters(), lr=0.001 , weight_decay=0.0001)

# Check if the GPU is available
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(f'Selected device: {device}')

#Send model to device
model.to(device)

Selected device: cpu


Net(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (drop): Dropout(p=0.25, inplace=False)
  (soft): Softmax(dim=None)
)

In [11]:
train_loss_1, train_loss_2, valid_loss = model.training_cycle(device,
                                train_loader,
                                valid_loader,
                                loss_fn,
                                optimizer,
                                max_epochs,
                                keep_model=False,
                                verbose=True)

  0%|                                                                                         | 0/30 [00:00<?, ?it/s]


NameError: name 'train_epoch' is not defined

In [ ]:
# saveModel(model)

In [ ]:
model.load_state_dict(torch.load("./Modelos_Fiteados/REG/3C_L1.pth"))

In [ ]:
# np.savetxt('./Modelos_Fiteados/losses/REG/Nuevas_loss/errorTrain_3CL1',train_loss_1)
np.savetxt('./Modelos_Fiteados/losses/REG/Nuevas_loss/errorTrain_3CL1',train_loss_2)
np.savetxt('./Modelos_Fiteados/losses/REG/Nuevas_loss/errorVal_3CL1',valid_loss)

# train_loss = np.loadtxt('./Modelos_Fiteados/losses/REG/trainloss_2C_L1')
# valid_loss = np.loadtxt('./Modelos_Fiteados/losses/REG/validloss_2C_L1')

In [ ]:
# Plot losses by epoch
plt.figure(figsize=(7,5))
# plt.plot(train_loss_1, linestyle='--', marker='o',  label='Train 1 loss', c='mediumaquamarine')
plt.plot(train_loss_2, linestyle='--', marker='o',  label='Train 2 loss', c='green')
plt.plot(valid_loss, linestyle='--', marker='o', label='Valid loss', c='darkviolet')
plt.xlabel('Época',fontsize=15)
plt.ylabel('Loss',fontsize=15)
plt.title("Función de costo ...........",fontsize=15)
plt.grid()
plt.legend()
plt.savefig('./Modelos_Fiteados/Imagenes/REG/errores_........_.png', format='png')
plt.show()

In [ ]:
prediccion = []

model.to(device)

with torch.no_grad():
    for data in test_loader:
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        
        # forward + backward + optimize
        outputs = model(inputs, "Test")
        outputs = outputs.detach().cpu().numpy()
        labels = labels.detach().cpu().numpy()
        prediccion.append([outputs[0][0],labels[0]])
        #print(outputs)
prediccion = np.array(prediccion)

In [ ]:
# np.savetxt('./Modelos_Fiteados/REG/predic_3C_L1',prediccion)

prediccion = np.loadtxt('./Modelos_Fiteados/REG/predic_3C_L1')

In [ ]:
# plt.figure(figsize=(7,5))
plt.scatter(prediccion[:,1],prediccion[:,0], marker="+", s=1, alpha=.2, c='blueviolet')
plt.plot(np.linspace(-60, 60, num=100),np.linspace(-60, 60, num=100), linestyle='--', c='orange')
plt.xlabel('Valor REAL',fontsize=15)
plt.ylabel('Valor PREDICHO',fontsize=15)
plt.title("Dispersión de valores predichos",fontsize=15)
plt.xlim(-60,60)
plt.savefig('./Modelos_Fiteados/Imagenes/REG/disper_3C_L1.png', format='png')
plt.show()

In [ ]:
# plt.figure(figsize=(8, 8))
# plt.scatter(prediccion[:,0],[i for i in range(len(prediccion[:,0]))],alpha=0.05)
# plt.scatter(prediccion[:,1],[i for i in range(len(prediccion[:,1]))],alpha=0.05)
# plt.show()

### Vamos a discretizar las cosas asi vemos una especie de CM

In [ ]:
prediccion = np.loadtxt('./Modelos_Fiteados/REG/predic_2C_L1')

In [ ]:
i=0
for i in range(len(prediccion)):
    if -60<= prediccion[i,0]< -45:
        prediccion[i,0]=0
    elif -45 <= prediccion[i,0] < -30:
        prediccion[i,0]=1
    elif -30<= prediccion[i,0] < -15:
        prediccion[i,0]=2
    elif -15 <= prediccion[i,0] < 0:
        prediccion[i,0]=3
    elif 0 <= prediccion[i,0] <15:
        prediccion[i,0]=4
    elif 15 <= prediccion[i,0] <30:
        prediccion[i,0]=5
    elif 30 <= prediccion[i,0] <45:
        prediccion[i,0]=6
    else:
        prediccion[i,0]=7        
        
i=0
for i in range(len(prediccion)):
    if -60 <= prediccion[i,1]< -45:
        prediccion[i,1]=0
    elif -45 <= prediccion[i,1] < -30:
        prediccion[i,1]=1
    elif -30<= prediccion[i,1] < -15:
        prediccion[i,1]=2
    elif -15 <= prediccion[i,1] < 0:
        prediccion[i,1]=3
    elif 0 <= prediccion[i,1] <15:
        prediccion[i,1]=4
    elif 15 <= prediccion[i,1] <30:
        prediccion[i,1]=5
    elif 30 <= prediccion[i,1] <45:
        prediccion[i,1]=6
    else:
        prediccion[i,1]=7

In [ ]:
label = prediccion[:,0]
pred = prediccion[:,1]

In [ ]:
def confussion_plot(real, prediction):
    categories = ['Clase 1', 'Clase 2', 'Clase 3', 'Clase 4', 'Clase 5', 'Clase 6', 'Clase 7', 'Clase 8']

    array = confusion_matrix(real, prediction)#, normalize="true")
    df_cm = pd.DataFrame(array, index = [i for i in categories], columns = [i for i in categories])
    
    fig = plt.figure(figsize=(8,7))
    sn.heatmap(df_cm, annot=True, fmt='.2f', cmap='viridis')
    plt.ylabel("Etiqueta",fontsize=15)
    plt.xlabel("Predicción",fontsize=15)
    plt.title("Matríz de confusión",fontsize=15)
    plt.savefig('./Modelos_Fiteados/Imagenes/REG/CM_2C_L1.png', format='png')
    plt.show()

In [ ]:
confussion_plot(label, pred)

In [ ]:
right = 0
test_acc = 0

for i in range(len(label)):
    right = np.sum(label[i]==pred[i])
    test_acc = test_acc + right

print(f"The obtained accuracy is {test_acc/len(label)*100} %")

en 5 clases a ver

In [ ]:
prediccion = np.loadtxt('./Modelos_Fiteados/REG/predic_3C_L1')

In [ ]:
i=0
for i in range(len(prediccion)):
    if -60<= prediccion[i,0]< -36:
        prediccion[i,0]=0
    elif -36 <= prediccion[i,0] < -12:
        prediccion[i,0]=1
    elif -12<= prediccion[i,0] < 12:
        prediccion[i,0]=2
    elif 12 <= prediccion[i,0] < 36:
        prediccion[i,0]=3
    else:
        prediccion[i,0]=4        
        
i=0
for i in range(len(prediccion)):
    if -60 <= prediccion[i,1]< -36:
        prediccion[i,1]=0
    elif -36 <= prediccion[i,1] < -12:
        prediccion[i,1]=1
    elif -12<= prediccion[i,1] < 12:
        prediccion[i,1]=2
    elif 12 <= prediccion[i,1] < 36:
        prediccion[i,1]=3
    else:
        prediccion[i,1]=4

In [ ]:
label = prediccion[:,0]
pred = prediccion[:,1]

In [ ]:
def confussion_plot(real, prediction):
    categories = ['Clase 1', 'Clase 2', 'Clase 3', 'Clase 4', 'Clase 5']

    array = confusion_matrix(real, prediction, normalize="true")
    df_cm = pd.DataFrame(array, index = [i for i in categories], columns = [i for i in categories])
    
    fig = plt.figure(figsize=(8,7))
    sn.heatmap(df_cm, annot=True, fmt='.2f', cmap='viridis')
    plt.ylabel("Etiqueta",fontsize=15)
    plt.xlabel("Predicción",fontsize=15)
    plt.title("Matríz de confusión",fontsize=15)
    plt.savefig('./Modelos_Fiteados/Imagenes/REG/MC3CL1_5_NORM.png', format='png')
    plt.show()

In [ ]:
confussion_plot(label, pred)

#### Veamos si me sale el grafico delas cosas pero con la recta ajustada

In [ ]:
a = -0.0064
b = 1.009

x = np.arange(-0.6,0.6, 0.001)
y = a + b*x

# plt.plot(x,y)
# plt.xlabel('x')
# plt.ylabel('y')
# plt.title('Lab DLS')
# plt.show()

In [ ]:
prediccion = np.loadtxt('./Modelos_Fiteados/REG/predic_')

In [ ]:
plt.figure(figsize=(7,5))
plt.scatter(prediccion[:,0],prediccion[:,1], marker="+", color='navy', s=5, alpha=.2)
plt.plot(np.linspace(-0.6, 0.6, num=100),np.linspace(-0.6, 0.6, num=100), linestyle='--', color='sandybrown')
plt.scatter(x,y, marker ='*' , color = 'darkviolet', s=5, alpha=0.5)
plt.xlabel('Valor predicho',fontsize=15)
plt.ylabel('Valor real',fontsize=15)
plt.title("Dispersión 2C_L1",fontsize=15)
plt.xlim(-0.8,0.8)
plt.legend(labels = ['Puntos','Identidad','Ajuste'])
# plt.savefig('./Modelos_Fiteados/Imagenes/REG/disper_3c_L1.png', format='png')
plt.show()